Notebook di elaborazione dei dati delle comunicazioni_sdi per analisi su Tableau

In [38]:
import pandas as pd
import re
from utils.functions import age_clustering, age_sort

Creazione del kpi "Percentuale di donne vittime del seguente reato per classe d'età negli ultimi 5 anni"

il kpi è stato creato nel seguente modo:
- popolazione residente in Italia nel 2022 (anno mediano)
- conteggio delle vittime per tipologia di reato comunicazione_sdi (delitti + codice rosso + reati spia)  e classe di età
- vittime/popolazione residente 2022

In [39]:
df_delitti_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/delitti_vittime.csv")
df_codice_rosso_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/codice_rosso_vittime.csv")
df_reati_spia_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/reati_spia_vittime.csv")
df_popolazione_italia_2022 = pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/resources/popolazione_residente_italia_2022.csv")

In [40]:
"""Mappatura in classi di età della popolazione residente in italia nel 2022"""
df_popolazione_italia_2022 = df_popolazione_italia_2022.drop(df_popolazione_italia_2022.index[-1]) #rimuovo la riga dei totali
df_popolazione_italia_2022["classi_eta"] = df_popolazione_italia_2022["Età"].apply(age_clustering)
df_popolazione_italia_2022_c = df_popolazione_italia_2022[["Totale maschi", "Totale femmine", "classi_eta"]]
df_residenti_classi_eta_2022 = df_popolazione_italia_2022_c.groupby("classi_eta").sum().reset_index()
df_residenti_classi_eta_2022

,classi_eta,Totale maschi,Totale femmine
0,0-13,3555806,3359301
1,14-17,1188841,1114966
2,18-24,2121760,1938579
3,25-34,3182043,3030951
4,35-44,3652999,3627957
5,45-54,4669124,4747983
6,55-64,4285158,4503261
7,65 e oltre,6163225,7888179


In [41]:
"Aggregazioni per classi di età delle vittime nei dataset SDI"
df_delitti_f_classi_eta = df_delitti_vittime.groupby(["eta_alla_data_del_reato_vittima", "delitto"])["valore"].sum().reset_index()
df_delitti_f_classi_eta = df_delitti_f_classi_eta.rename(columns={"delitto": "descrizione_reato"})
df_delitti_f_classi_eta["descrizione_reato"] = df_delitti_f_classi_eta["descrizione_reato"].str.replace(r"^\d+\.\s*", "", regex=True).str.upper()
#print(df_delitti_f_classi_eta)
df_codice_rosso_f_classi_eta = df_codice_rosso_vittime.groupby(["eta_alla_data_del_reato_vittima", "descrizione_reato"])["valore"].sum().reset_index()
#print(df_codice_rosso_f_classi_eta)
df_reati_spia_f_classi_eta = df_reati_spia_vittime.groupby(["eta_alla_data_del_reato_vittima", "descrizione_reato"])["valore"].sum().reset_index()
#print(df_reati_spia_f_classi_eta)
print(df_delitti_f_classi_eta.shape, df_codice_rosso_f_classi_eta.shape, df_reati_spia_f_classi_eta.shape)
df_comunicazioni_sdi = pd.concat([df_delitti_f_classi_eta, df_codice_rosso_f_classi_eta, df_reati_spia_f_classi_eta], axis=0)
print(df_comunicazioni_sdi)

(40, 3) (29, 3) (16, 3)
   eta_alla_data_del_reato_vittima  \
0                             0-13   
1                             0-13   
2                             0-13   
3                             0-13   
4                             0-13   
..                             ...   
11                           45-54   
12                           55-64   
13                           55-64   
14                      65 e oltre   
15                      65 e oltre   

                               descrizione_reato  valore  
0                                        MINACCE    1480  
1                              VIOLENZE SESSUALI    2330  
2                                TENTATI OMICIDI      24  
3                                 LESIONI DOLOSE    2535  
4                                       PERCOSSE     972  
..                                           ...     ...  
11  MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI   23338  
12                              ATTI PERSECUTOR

In [42]:
df_comunicazioni_sdi[df_comunicazioni_sdi["descrizione_reato"] == "COSTRIZIONE O INDUZIONE AL MATRIMONIO"]


,eta_alla_data_del_reato_vittima,descrizione_reato,valore
0,0-13,COSTRIZIONE O INDUZIONE AL MATRIMONIO,3
4,14-17,COSTRIZIONE O INDUZIONE AL MATRIMONIO,29
8,18-24,COSTRIZIONE O INDUZIONE AL MATRIMONIO,48
12,25-34,COSTRIZIONE O INDUZIONE AL MATRIMONIO,11
19,45-54,COSTRIZIONE O INDUZIONE AL MATRIMONIO,2


In [43]:
df_comunicazioni_sdi[df_comunicazioni_sdi["descrizione_reato"] == "COSTRIZIONE O INDUZIONE AL MATRIMONIO"]
nuova_riga = pd.DataFrame([{"eta_alla_data_del_reato_vittima": "35-44", "descrizione_reato": "COSTRIZIONE O INDUZIONE AL MATRIMONIO", "valore": 0}])
nuova_riga_2 = pd.DataFrame([{"eta_alla_data_del_reato_vittima": "55-64", "descrizione_reato": "COSTRIZIONE O INDUZIONE AL MATRIMONIO", "valore": 0}])
nuova_riga_3 = pd.DataFrame([{"eta_alla_data_del_reato_vittima": "65 e oltre", "descrizione_reato": "COSTRIZIONE O INDUZIONE AL MATRIMONIO", "valore": 0}])
df_comunicazioni_sdi = pd.concat([df_comunicazioni_sdi, nuova_riga, nuova_riga_2, nuova_riga_3], axis=0)
df_comunicazioni_sdi[df_comunicazioni_sdi["descrizione_reato"] == "COSTRIZIONE O INDUZIONE AL MATRIMONIO"]


,eta_alla_data_del_reato_vittima,descrizione_reato,valore
0,0-13,COSTRIZIONE O INDUZIONE AL MATRIMONIO,3
4,14-17,COSTRIZIONE O INDUZIONE AL MATRIMONIO,29
8,18-24,COSTRIZIONE O INDUZIONE AL MATRIMONIO,48
12,25-34,COSTRIZIONE O INDUZIONE AL MATRIMONIO,11
19,45-54,COSTRIZIONE O INDUZIONE AL MATRIMONIO,2
0,35-44,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0
0,55-64,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0
0,65 e oltre,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0


In [44]:
"""merge con popolazione residente in italia 2022"""
df_comunicazioni_sdi_f = df_comunicazioni_sdi.merge(df_residenti_classi_eta_2022, how="left", left_on="eta_alla_data_del_reato_vittima", right_on="classi_eta")
df_comunicazioni_sdi_f["p_vittime"]= round(df_comunicazioni_sdi_f["valore"] / (df_comunicazioni_sdi_f["Totale femmine"])*100,2)
df_comunicazioni_sdi_f["ordinamento_classe_eta"] = df_comunicazioni_sdi_f["eta_alla_data_del_reato_vittima"].apply(age_sort)
df_comunicazioni_sdi_f = df_comunicazioni_sdi_f.sort_values(by=["ordinamento_classe_eta", "p_vittime"], ascending=[True, False])
#df_comunicazioni_sdi_f_max=df_comunicazioni_sdi_f.loc[df_comunicazioni_sdi_f.groupby("eta_alla_data_del_reato_vittima")["p_vittime"].idxmax()]
#df_comunicazioni_sdi_f_max

In [45]:
df_totale_vittime_classi_eta = df_comunicazioni_sdi.groupby("eta_alla_data_del_reato_vittima")["valore"].sum()
print(df_totale_vittime_classi_eta) 

eta_alla_data_del_reato_vittima
0-13           13373
14-17          20953
18-24          53984
25-34         105085
35-44         134769
45-54         122829
55-64          63719
65 e oltre     44822
Name: valore, dtype: int64


In [46]:
df_totale_vittime_classi_eta = df_comunicazioni_sdi.merge(df_totale_vittime_classi_eta, how="left", on="eta_alla_data_del_reato_vittima").rename(columns={"valore_y": "totale_vittime_classe_eta", "valore_x": "valore"})
df_totale_vittime_classi_eta

,eta_alla_data_del_reato_vittima,descrizione_reato,valore,totale_vittime_classe_eta
0,0-13,MINACCE,1480,13373
1,0-13,VIOLENZE SESSUALI,2330,13373
2,0-13,TENTATI OMICIDI,24,13373
3,0-13,LESIONI DOLOSE,2535,13373
4,0-13,PERCOSSE,972,13373
...,...,...,...,...
83,65 e oltre,ATTI PERSECUTORI,3748,44822
84,65 e oltre,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,11167,44822
85,35-44,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,134769
86,55-64,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,63719


In [47]:
df_totale_vittime_classi_eta["p_vittime_su_tot_reati"]= round(df_totale_vittime_classi_eta["valore"] / df_totale_vittime_classi_eta["totale_vittime_classe_eta"]*100,2)
df_totale_vittime_classi_eta["ordinamento_classe_eta"] = df_totale_vittime_classi_eta["eta_alla_data_del_reato_vittima"].apply(age_sort)
df_totale_vittime_classi_eta

,eta_alla_data_del_reato_vittima,descrizione_reato,valore,totale_vittime_classe_eta,p_vittime_su_tot_reati,ordinamento_classe_eta
0,0-13,MINACCE,1480,13373,11.07,0
1,0-13,VIOLENZE SESSUALI,2330,13373,17.42,0
2,0-13,TENTATI OMICIDI,24,13373,0.18,0
3,0-13,LESIONI DOLOSE,2535,13373,18.96,0
4,0-13,PERCOSSE,972,13373,7.27,0
...,...,...,...,...,...,...
83,65 e oltre,ATTI PERSECUTORI,3748,44822,8.36,7
84,65 e oltre,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,11167,44822,24.91,7
85,35-44,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,134769,0.00,4
86,55-64,COSTRIZIONE O INDUZIONE AL MATRIMONIO,0,63719,0.00,6


In [48]:
df_comunicazioni_sdi_vittime_f = df_totale_vittime_classi_eta.sort_values(by=["ordinamento_classe_eta", "p_vittime_su_tot_reati"], ascending=[True, False])
#df_comunicazioni_sdi_vittime_f_max=df_comunicazioni_sdi_vittime_f.loc[df_comunicazioni_sdi_vittime_f.groupby("eta_alla_data_del_reato_vittima")["p_vittime_su_tot_reati"].idxmax()]
#df_comunicazioni_sdi_vittime_f_max
df_comunicazioni_sdi_vittime_f.to_csv("df_comunicazioni_sdi_vittime_f.csv", index=False)

In [49]:
df_comunicazioni_sdi_vittime_f

,eta_alla_data_del_reato_vittima,descrizione_reato,valore,totale_vittime_classe_eta,p_vittime_su_tot_reati,ordinamento_classe_eta
70,0-13,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,4885,13373,36.53,0
3,0-13,LESIONI DOLOSE,2535,13373,18.96,0
1,0-13,VIOLENZE SESSUALI,2330,13373,17.42,0
0,0-13,MINACCE,1480,13373,11.07,0
4,0-13,PERCOSSE,972,13373,7.27,0
...,...,...,...,...,...,...
36,65 e oltre,VIOLENZE SESSUALI,351,44822,0.78,7
37,65 e oltre,TENTATI OMICIDI,151,44822,0.34,7
67,65 e oltre,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,20,44822,0.04,7
66,65 e oltre,DEFORMAZIONE DELL'ASPETTO DELLA PERSONA MEDIAN...,2,44822,0.00,7


In [50]:
residenti_2022_province =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/resources/POSAS_2022_it_Province.csv", sep=";", header=1)
residenti_2022_province

,Codice provincia,Provincia,Età,Celibi,Coniugati,Divorziati,Vedovi,Uniti civilmente,Maschi già in unione civile (per scioglimento unione),Maschi già in unione civile (per decesso del partner),Totale maschi,Nubili,Coniugate,Divorziate,Vedove,Unite civilmente,Femmine già in unione civile (per scioglimento unione),Femmine già in unione civile (per decesso del partner),Totale femmine,Totale
0,84,Agrigento,0,1540,0,0,0,0,0,0,1540,1446,0,0,0,0,0,0,1446,2986
1,84,Agrigento,1,1512,0,0,0,0,0,0,1512,1516,0,0,0,0,0,0,1516,3028
2,84,Agrigento,2,1606,0,0,0,0,0,0,1606,1532,0,0,0,0,0,0,1532,3138
3,84,Agrigento,3,1710,0,0,0,0,0,0,1710,1547,0,0,0,0,0,0,1547,3257
4,84,Agrigento,4,1666,0,0,0,0,0,0,1666,1684,0,0,0,0,0,0,1684,3350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10909,56,Viterbo,97,3,8,0,25,0,0,0,36,11,2,4,80,0,0,0,97,133
10910,56,Viterbo,98,0,6,0,6,0,0,0,12,8,2,1,83,0,0,0,94,106
10911,56,Viterbo,99,0,3,0,6,0,0,0,9,2,0,1,61,0,0,0,64,73
10912,56,Viterbo,100,1,5,0,13,0,0,0,19,5,1,0,58,0,0,0,64,83


In [51]:
residenti_2022_province['classi_eta']= residenti_2022_province['Età'].apply(age_clustering)

In [52]:
residenti_2022_province_grouped = (
    residenti_2022_province
    .groupby(['Codice provincia', 'Provincia', 'classi_eta'])[['Totale maschi', 'Totale femmine']]
    .sum()
    .reset_index()
)

In [53]:
residenti_2022_province_grouped

,Codice provincia,Provincia,classi_eta,Totale maschi,Totale femmine
0,1,Torino,0-13,126758,119320
1,1,Torino,14-17,42063,39222
2,1,Torino,18-24,74227,67931
3,1,Torino,25-34,113970,106727
4,1,Torino,35-44,129539,130624
...,...,...,...,...,...
851,111,Sud Sardegna,25-34,16385,15175
852,111,Sud Sardegna,35-44,21571,20300
853,111,Sud Sardegna,45-54,27366,26536
854,111,Sud Sardegna,55-64,27461,27737


In [54]:
residenti_2022_province_grouped['PROVINCIA_CLEAN'] = residenti_2022_province_grouped['Provincia'].str.upper()

In [55]:
regioni_province =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/resources/province_sdi_istat.csv", sep=",", header=0)
regioni_province

,provincia,provinciauts_corretto,codice_regione,codice_provincia_storico,codice_provinciauts,ripartizione_geografica,regione,provincia_uts,flag_tipo_uts,sigla_automobilistica
0,AGRIGENTO,AGRIGENTO,19,84,84,Isole,Sicilia,Agrigento,4,AG
1,ALESSANDRIA,ALESSANDRIA,1,6,6,Nord-ovest,Piemonte,Alessandria,1,AL
2,ANCONA,ANCONA,11,42,42,Centro,Marche,Ancona,1,AN
3,AOSTA,VALLE D'AOSTA/VALLÃ‰E D'AOSTE,2,7,7,Nord-ovest,Valle d'Aosta/Vallée d'Aoste,Valle d'Aosta/Vallée d'Aoste,1,AO
4,AREZZO,AREZZO,9,51,51,Centro,Toscana,Arezzo,1,AR
...,...,...,...,...,...,...,...,...,...,...
102,VERCELLI,VERCELLI,1,2,2,Nord-ovest,Piemonte,Vercelli,1,VC
103,VERONA,VERONA,5,23,23,Nord-est,Veneto,Verona,1,VR
104,VIBO VALENTIA,VIBO VALENTIA,18,102,102,Sud,Calabria,Vibo Valentia,1,VV
105,VICENZA,VICENZA,5,24,24,Nord-est,Veneto,Vicenza,1,VI


In [56]:
df1 =residenti_2022_province_grouped
df2 =regioni_province
residenti_2022_province_grouped = pd.merge(df1, df2, left_on='Codice provincia', right_on='codice_provincia_storico', how='left')
residenti_2022_province_grouped

,Codice provincia,Provincia,classi_eta,Totale maschi,Totale femmine,PROVINCIA_CLEAN,provincia,provinciauts_corretto,codice_regione,codice_provincia_storico,codice_provinciauts,ripartizione_geografica,regione,provincia_uts,flag_tipo_uts,sigla_automobilistica
0,1,Torino,0-13,126758,119320,TORINO,TORINO,TORINO,1.0,1.0,201.0,Nord-ovest,Piemonte,Torino,3.0,TO
1,1,Torino,14-17,42063,39222,TORINO,TORINO,TORINO,1.0,1.0,201.0,Nord-ovest,Piemonte,Torino,3.0,TO
2,1,Torino,18-24,74227,67931,TORINO,TORINO,TORINO,1.0,1.0,201.0,Nord-ovest,Piemonte,Torino,3.0,TO
3,1,Torino,25-34,113970,106727,TORINO,TORINO,TORINO,1.0,1.0,201.0,Nord-ovest,Piemonte,Torino,3.0,TO
4,1,Torino,35-44,129539,130624,TORINO,TORINO,TORINO,1.0,1.0,201.0,Nord-ovest,Piemonte,Torino,3.0,TO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,111,Sud Sardegna,25-34,16385,15175,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
860,111,Sud Sardegna,35-44,21571,20300,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,111,Sud Sardegna,45-54,27366,26536,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
862,111,Sud Sardegna,55-64,27461,27737,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
residenti_2022_province_grouped[residenti_2022_province_grouped['regione'].isnull()]['Provincia'].unique()

array(['Sud Sardegna'], dtype=object)

In [58]:
regioni_province[regioni_province['regione']=='Sardegna']


,provincia,provinciauts_corretto,codice_regione,codice_provincia_storico,codice_provinciauts,ripartizione_geografica,regione,provincia_uts,flag_tipo_uts,sigla_automobilistica
18,CAGLIARI,CAGLIARI,20,92,292,Isole,Sardegna,Cagliari,3,CA
60,NUORO,NUORO,20,91,91,Isole,Sardegna,Nuoro,1,NU
61,ORISTANO,ORISTANO,20,95,95,Isole,Sardegna,Oristano,1,OR
85,SASSARI,SASSARI,20,90,90,Isole,Sardegna,Sassari,1,SS


In [59]:
residenti_2022_province_grouped[residenti_2022_province_grouped['Provincia']=='Sud Sardegna']

,Codice provincia,Provincia,classi_eta,Totale maschi,Totale femmine,PROVINCIA_CLEAN,provincia,provinciauts_corretto,codice_regione,codice_provincia_storico,codice_provinciauts,ripartizione_geografica,regione,provincia_uts,flag_tipo_uts,sigla_automobilistica
856,111,Sud Sardegna,0-13,16044,14979,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,111,Sud Sardegna,14-17,5640,5130,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,111,Sud Sardegna,18-24,10269,9289,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
859,111,Sud Sardegna,25-34,16385,15175,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
860,111,Sud Sardegna,35-44,21571,20300,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,111,Sud Sardegna,45-54,27366,26536,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
862,111,Sud Sardegna,55-64,27461,27737,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863,111,Sud Sardegna,65 e oltre,42477,50819,SUD SARDEGNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
df_delitti_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/delitti_vittime.csv")
df_delitti_vittime

,anno,provincia,delitto,eta_alla_data_del_reato_vittima,valore,provinciauts_corretto,codice_regione,codice_provincia_storico,codice_provinciauts,ripartizione_geografica,regione,provincia_uts,flag_tipo_uts,sigla_automobilistica
0,2019,AGRIGENTO,5. TENTATI OMICIDI,0-13,0,AGRIGENTO,19.0,84.0,84.0,Isole,Sicilia,Agrigento,4.0,AG
1,2020,AGRIGENTO,5. TENTATI OMICIDI,0-13,1,AGRIGENTO,19.0,84.0,84.0,Isole,Sicilia,Agrigento,4.0,AG
2,2021,AGRIGENTO,5. TENTATI OMICIDI,0-13,0,AGRIGENTO,19.0,84.0,84.0,Isole,Sicilia,Agrigento,4.0,AG
3,2022,AGRIGENTO,5. TENTATI OMICIDI,0-13,0,AGRIGENTO,19.0,84.0,84.0,Isole,Sicilia,Agrigento,4.0,AG
4,2023,AGRIGENTO,5. TENTATI OMICIDI,0-13,0,AGRIGENTO,19.0,84.0,84.0,Isole,Sicilia,Agrigento,4.0,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23155,2020,NON LOCALIZZATA,12. VIOLENZE SESSUALI,45-54,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23156,2021,NON LOCALIZZATA,12. VIOLENZE SESSUALI,45-54,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23157,2022,NON LOCALIZZATA,12. VIOLENZE SESSUALI,45-54,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23158,2023,NON LOCALIZZATA,12. VIOLENZE SESSUALI,45-54,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df_delitti_vittime_grouped = (
    df_delitti_vittime
    .groupby(['provinciauts_corretto', 'delitto', 'eta_alla_data_del_reato_vittima', 'codice_provincia_storico'])[['valore']]
    .sum()
    .reset_index()
)
df_delitti_vittime_grouped

,provinciauts_corretto,delitto,eta_alla_data_del_reato_vittima,codice_provincia_storico,valore
0,AGRIGENTO,10. MINACCE,0-13,84.0,16
1,AGRIGENTO,10. MINACCE,14-17,84.0,31
2,AGRIGENTO,10. MINACCE,18-24,84.0,114
3,AGRIGENTO,10. MINACCE,25-34,84.0,331
4,AGRIGENTO,10. MINACCE,35-44,84.0,418
...,...,...,...,...,...
3828,VITERBO,9. PERCOSSE,25-34,56.0,27
3829,VITERBO,9. PERCOSSE,35-44,56.0,47
3830,VITERBO,9. PERCOSSE,45-54,56.0,31
3831,VITERBO,9. PERCOSSE,55-64,56.0,25


In [62]:
df1 = df_delitti_vittime_grouped[[ 'provinciauts_corretto', 'delitto', 'eta_alla_data_del_reato_vittima', 'valore', 'codice_provincia_storico']].copy()
df2 =residenti_2022_province_grouped [[	'classi_eta',	'Totale maschi','Totale femmine', 'codice_provincia_storico']].copy()
df_delitti_vittime_prov = pd.merge(df1, df2, left_on=['codice_provincia_storico','eta_alla_data_del_reato_vittima'], right_on=['codice_provincia_storico','classi_eta'], how='left')

In [63]:
df_delitti_vittime_prov.rename(columns={'delitto':'descrizione_reato'}, inplace=True)


In [64]:
df_delitti_vittime_prov

,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,valore,codice_provincia_storico,classi_eta,Totale maschi,Totale femmine
0,AGRIGENTO,10. MINACCE,0-13,16,84.0,0-13,25180,23799
1,AGRIGENTO,10. MINACCE,14-17,31,84.0,14-17,9069,8502
2,AGRIGENTO,10. MINACCE,18-24,114,84.0,18-24,17129,15799
3,AGRIGENTO,10. MINACCE,25-34,331,84.0,25-34,23951,22871
4,AGRIGENTO,10. MINACCE,35-44,418,84.0,35-44,24983,24825
...,...,...,...,...,...,...,...,...
3862,VITERBO,9. PERCOSSE,25-34,27,56.0,25-34,16113,14825
3863,VITERBO,9. PERCOSSE,35-44,47,56.0,35-44,19750,19492
3864,VITERBO,9. PERCOSSE,45-54,31,56.0,45-54,24733,25062
3865,VITERBO,9. PERCOSSE,55-64,25,56.0,55-64,23610,24064


In [65]:
# df_delitti_vittime_prov['perc_donne_vittime']=df_delitti_vittime_prov['valore']/df_delitti_vittime_prov['Totale femmine']*1000
# x= df_delitti_vittime_prov[df_delitti_vittime_prov['valore']!=0]['perc_donne_vittime'].max()
# y= df_delitti_vittime_prov[df_delitti_vittime_prov['valore']!=0]['perc_donne_vittime'].min()
# print(x, y)

In [66]:
"""calcoliamo la stessa cosa anche per le altre vittime, poi uniamo il tutto categorizzando per fogli"""
df_codice_rosso_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/codice_rosso_vittime.csv")
df_reati_spia_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/reati_spia_vittime.csv")

In [67]:
df_codice_rosso_vittime_grouped = (
    df_codice_rosso_vittime
    .groupby(['provinciauts_corretto', 'descrizione_reato', 'eta_alla_data_del_reato_vittima', 'codice_provincia_storico'])[['valore']]
    .sum()
    .reset_index()
)
df_codice_rosso_vittime_grouped

,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,codice_provincia_storico,valore
0,AGRIGENTO,COSTRIZIONE O INDUZIONE AL MATRIMONIO,45-54,84.0,1
1,AGRIGENTO,DEFORMAZIONE DELL'ASPETTO DELLA PERSONA MEDIAN...,25-34,84.0,2
2,AGRIGENTO,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,0-13,84.0,5
3,AGRIGENTO,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,14-17,84.0,6
4,AGRIGENTO,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,18-24,84.0,10
...,...,...,...,...,...
1484,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,25-34,56.0,17
1485,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,35-44,56.0,24
1486,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,45-54,56.0,18
1487,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,55-64,56.0,4


In [68]:
df_reati_spia_vittime_grouped = (
    df_reati_spia_vittime
    .groupby(['provinciauts_corretto', 'descrizione_reato', 'eta_alla_data_del_reato_vittima', 'codice_provincia_storico'])[['valore']]
    .sum()
    .reset_index()
)
df_reati_spia_vittime_grouped

,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,codice_provincia_storico,valore
0,AGRIGENTO,ATTI PERSECUTORI,0-13,84.0,8
1,AGRIGENTO,ATTI PERSECUTORI,14-17,84.0,22
2,AGRIGENTO,ATTI PERSECUTORI,18-24,84.0,79
3,AGRIGENTO,ATTI PERSECUTORI,25-34,84.0,140
4,AGRIGENTO,ATTI PERSECUTORI,35-44,84.0,223
...,...,...,...,...,...
1685,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,25-34,56.0,128
1686,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,35-44,56.0,170
1687,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,45-54,56.0,145
1688,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,55-64,56.0,66


In [69]:
df1 = df_codice_rosso_vittime_grouped[[ 'provinciauts_corretto', 'descrizione_reato', 'eta_alla_data_del_reato_vittima', 'valore', 'codice_provincia_storico']].copy()
df2 =residenti_2022_province_grouped [[	'classi_eta',	'Totale maschi','Totale femmine', 'codice_provincia_storico']].copy()
df_codice_rosso_vittime_prov = pd.merge(df1, df2, left_on=['codice_provincia_storico','eta_alla_data_del_reato_vittima'], right_on=['codice_provincia_storico','classi_eta'], how='left')
df_codice_rosso_vittime_prov

,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,valore,codice_provincia_storico,classi_eta,Totale maschi,Totale femmine
0,AGRIGENTO,COSTRIZIONE O INDUZIONE AL MATRIMONIO,45-54,1,84.0,45-54,30447,32051
1,AGRIGENTO,DEFORMAZIONE DELL'ASPETTO DELLA PERSONA MEDIAN...,25-34,2,84.0,25-34,23951,22871
2,AGRIGENTO,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,0-13,5,84.0,0-13,25180,23799
3,AGRIGENTO,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,14-17,6,84.0,14-17,9069,8502
4,AGRIGENTO,DIFFUSIONE ILLECITA DI IMMAGINI O VIDEO SESSUA...,18-24,10,84.0,18-24,17129,15799
...,...,...,...,...,...,...,...,...
1497,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,25-34,17,56.0,25-34,16113,14825
1498,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,35-44,24,56.0,35-44,19750,19492
1499,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,45-54,18,56.0,45-54,24733,25062
1500,VITERBO,VIOLAZIONE DEI PROVVEDIMENTI DI ALLONTANAMENTO...,55-64,4,56.0,55-64,23610,24064


In [70]:
df1 = df_reati_spia_vittime_grouped[['provinciauts_corretto', 'descrizione_reato', 'eta_alla_data_del_reato_vittima', 'valore', 'codice_provincia_storico']].copy()
df2 =residenti_2022_province_grouped [[	'classi_eta',	'Totale maschi','Totale femmine', 'codice_provincia_storico']].copy()
df_reati_spia_vittime_prov = pd.merge(df1, df2, left_on=['codice_provincia_storico','eta_alla_data_del_reato_vittima'], right_on=['codice_provincia_storico','classi_eta'], how='left')
df_reati_spia_vittime_prov

,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,valore,codice_provincia_storico,classi_eta,Totale maschi,Totale femmine
0,AGRIGENTO,ATTI PERSECUTORI,0-13,8,84.0,0-13,25180,23799
1,AGRIGENTO,ATTI PERSECUTORI,14-17,22,84.0,14-17,9069,8502
2,AGRIGENTO,ATTI PERSECUTORI,18-24,79,84.0,18-24,17129,15799
3,AGRIGENTO,ATTI PERSECUTORI,25-34,140,84.0,25-34,23951,22871
4,AGRIGENTO,ATTI PERSECUTORI,35-44,223,84.0,35-44,24983,24825
...,...,...,...,...,...,...,...,...
1700,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,25-34,128,56.0,25-34,16113,14825
1701,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,35-44,170,56.0,35-44,19750,19492
1702,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,45-54,145,56.0,45-54,24733,25062
1703,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,55-64,66,56.0,55-64,23610,24064


In [71]:
df_reati_spia_vittime_prov[df_reati_spia_vittime_prov['provinciauts_corretto'].isnull()] #sono i non localizzati

,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,valore,codice_provincia_storico,classi_eta,Totale maschi,Totale femmine


In [72]:
frames = [df_delitti_vittime_prov, df_codice_rosso_vittime_prov, df_reati_spia_vittime_prov]

result = pd.concat(frames)
result


,provinciauts_corretto,descrizione_reato,eta_alla_data_del_reato_vittima,valore,codice_provincia_storico,classi_eta,Totale maschi,Totale femmine
0,AGRIGENTO,10. MINACCE,0-13,16,84.0,0-13,25180,23799
1,AGRIGENTO,10. MINACCE,14-17,31,84.0,14-17,9069,8502
2,AGRIGENTO,10. MINACCE,18-24,114,84.0,18-24,17129,15799
3,AGRIGENTO,10. MINACCE,25-34,331,84.0,25-34,23951,22871
4,AGRIGENTO,10. MINACCE,35-44,418,84.0,35-44,24983,24825
...,...,...,...,...,...,...,...,...
1700,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,25-34,128,56.0,25-34,16113,14825
1701,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,35-44,170,56.0,35-44,19750,19492
1702,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,45-54,145,56.0,45-54,24733,25062
1703,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,55-64,66,56.0,55-64,23610,24064


In [73]:
result_filtrato = result[result['provinciauts_corretto'].notnull()]
result_filtrato =result_filtrato [[	'provinciauts_corretto',	'descrizione_reato','valore',	'codice_provincia_storico',	'classi_eta',	'Totale maschi',	'Totale femmine']]
result_filtrato 

,provinciauts_corretto,descrizione_reato,valore,codice_provincia_storico,classi_eta,Totale maschi,Totale femmine
0,AGRIGENTO,10. MINACCE,16,84.0,0-13,25180,23799
1,AGRIGENTO,10. MINACCE,31,84.0,14-17,9069,8502
2,AGRIGENTO,10. MINACCE,114,84.0,18-24,17129,15799
3,AGRIGENTO,10. MINACCE,331,84.0,25-34,23951,22871
4,AGRIGENTO,10. MINACCE,418,84.0,35-44,24983,24825
...,...,...,...,...,...,...,...
1700,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,128,56.0,25-34,16113,14825
1701,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,170,56.0,35-44,19750,19492
1702,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,145,56.0,45-54,24733,25062
1703,VITERBO,MALTRATTAMENTI CONTRO FAMILIARI E CONVIVENTI,66,56.0,55-64,23610,24064


In [74]:
result_filtrato.to_csv("reati_donne_per_anno_provincia_2019-24.csv", index=False)

## Calcolo Tasso reati di genere regionale

In [79]:
residenti_regioni_2022 = pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/resources/POSAS_2022_it_Regioni.csv", sep=";", header=1)
residenti_regioni_2022
residenti_2022_regione_grouped = (
    residenti_regioni_2022
    .groupby(['Codice regione', 'Regione'])[['Totale maschi', 'Totale femmine']]
    .sum()
    .reset_index()
)
residenti_2022_regione_grouped

,Codice regione,Regione,Totale maschi,Totale femmine
0,1,Piemonte,4147690,4365010
1,2,Valle d'Aosta/Vallée d'Aoste,120826,125894
2,3,Lombardia,9763056,10122952
3,4,Trentino-Alto Adige/Südtirol,1060938,1086210
4,5,Veneto,4761486,4934004
5,6,Friuli-Venezia Giulia,1165408,1223886
6,7,Liguria,1451642,1566812
7,8,Emilia-Romagna,4327070,4523662
8,9,Toscana,3553612,3772770
9,10,Umbria,830390,887234


In [81]:
df_delitti_vittime =pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/delitti_vittime.csv")
df_delitti_vittime_grouped = (
    df_delitti_vittime
    .groupby(['codice_regione', 'regione'])[['valore']]
    .sum()
    .reset_index()
)
df_delitti_vittime_grouped

,codice_regione,regione,valore
0,1.0,Piemonte,25244
1,2.0,Valle d'Aosta/Vallée d'Aoste,690
2,3.0,Lombardia,56684
3,4.0,Trentino-Alto Adige/Südtirol,5214
4,5.0,Veneto,23802
5,6.0,Friuli-Venezia Giulia,6370
6,7.0,Liguria,10175
7,8.0,Emilia-Romagna,30724
8,9.0,Toscana,22256
9,10.0,Umbria,5725


In [90]:
df1 = residenti_2022_regione_grouped
df2 = df_delitti_vittime_grouped
df_delitti_regioni = pd.merge(df1, df2, left_on='Codice regione', right_on='codice_regione')
df_delitti_regioni['tasso_delitti_femminile'] = df_delitti_regioni['valore']/ df_delitti_regioni['Totale femmine']*1000
df_delitti_regioni

,Codice regione,Regione,Totale maschi,Totale femmine,codice_regione,regione,valore,tasso_delitti_femminile
0,1,Piemonte,4147690,4365010,1.0,Piemonte,25244,5.783263
1,2,Valle d'Aosta/Vallée d'Aoste,120826,125894,2.0,Valle d'Aosta/Vallée d'Aoste,690,5.480801
2,3,Lombardia,9763056,10122952,3.0,Lombardia,56684,5.599552
3,4,Trentino-Alto Adige/Südtirol,1060938,1086210,4.0,Trentino-Alto Adige/Südtirol,5214,4.800177
4,5,Veneto,4761486,4934004,5.0,Veneto,23802,4.824074
5,6,Friuli-Venezia Giulia,1165408,1223886,6.0,Friuli-Venezia Giulia,6370,5.204733
6,7,Liguria,1451642,1566812,7.0,Liguria,10175,6.494078
7,8,Emilia-Romagna,4327070,4523662,8.0,Emilia-Romagna,30724,6.791843
8,9,Toscana,3553612,3772770,9.0,Toscana,22256,5.899114
9,10,Umbria,830390,887234,10.0,Umbria,5725,6.452638


In [ ]:
df_delitti_vittime = pd.read_csv("/Users/venere/Documents/GitHub/dati_sdi_reati_genere/data/processing/comunicazioni_sdi/delitti_vittime.csv")
df_delitti_vittime_grouped = (
    df_delitti_vittime
    .groupby(['codice_regione', 'regione'])[['valore']]
    .sum()
    .reset_index()
)
df_delitti_vittime_grouped

In [91]:
df_delitti_regioni

,Codice regione,Regione,Totale maschi,Totale femmine,codice_regione,regione,valore,tasso_delitti_femminile
0,1,Piemonte,4147690,4365010,1.0,Piemonte,25244,5.783263
1,2,Valle d'Aosta/Vallée d'Aoste,120826,125894,2.0,Valle d'Aosta/Vallée d'Aoste,690,5.480801
2,3,Lombardia,9763056,10122952,3.0,Lombardia,56684,5.599552
3,4,Trentino-Alto Adige/Südtirol,1060938,1086210,4.0,Trentino-Alto Adige/Südtirol,5214,4.800177
4,5,Veneto,4761486,4934004,5.0,Veneto,23802,4.824074
5,6,Friuli-Venezia Giulia,1165408,1223886,6.0,Friuli-Venezia Giulia,6370,5.204733
6,7,Liguria,1451642,1566812,7.0,Liguria,10175,6.494078
7,8,Emilia-Romagna,4327070,4523662,8.0,Emilia-Romagna,30724,6.791843
8,9,Toscana,3553612,3772770,9.0,Toscana,22256,5.899114
9,10,Umbria,830390,887234,10.0,Umbria,5725,6.452638


 3 viste + 1 aggregata

## Calcolo Tasso reati di genere provinciale


## Calcolo Tasso reati per età provincia e tipologia di reato